# 🔄 使用 GitHub 模型的基本代理工作流程 (.NET)

## 📋 工作流程編排教學

本筆記本展示如何使用 Microsoft Agent Framework for .NET 和 GitHub 模型構建複雜的 **代理工作流程**。您將學習如何創建多步驟的業務流程，讓 AI 代理通過結構化的編排模式協作完成複雜任務。

## 🎯 學習目標

### 🏗️ **工作流程架構基礎**
- **Workflow Builder**: 設計並編排複雜的多步驟 AI 流程
- **Agent Coordination**: 協調多個專業代理在工作流程中的合作
- **GitHub Models Integration**: 在工作流程中利用 GitHub 的 AI 模型推理服務
- **Visual Workflow Design**: 創建並可視化工作流程結構以便更好理解

### 🔄 **流程編排模式**
- **Sequential Processing**: 按邏輯順序鏈接多個代理任務
- **State Management**: 在工作流程階段間保持上下文和數據流
- **Error Handling**: 實現穩健的錯誤恢復和工作流程韌性
- **Performance Optimization**: 為企業級操作設計高效的工作流程

### 🏢 **企業工作流程應用**
- **Business Process Automation**: 自動化複雜的組織工作流程
- **Content Production Pipeline**: 包含審核和批准階段的編輯工作流程
- **Customer Service Automation**: 多步驟的客戶詢問解決方案
- **Data Processing Workflows**: 使用 AI 驅動的轉換進行 ETL 工作流程

## ⚙️ 先決條件與設置

### 📦 **所需的 NuGet 套件**

此工作流程演示使用了幾個關鍵的 .NET 套件：

```xml
<!-- Core AI Framework -->
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />

<!-- Agent Framework (Local Development) -->
<!-- Microsoft.Agents.AI.dll - Core agent abstractions -->
<!-- Microsoft.Agents.AI.OpenAI.dll - OpenAI/GitHub Models integration -->

<!-- Configuration and Environment -->
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

### 🔑 **GitHub 模型配置**

**環境設置 (.env 文件):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

**GitHub 模型訪問:**
1. 註冊 GitHub 模型（目前處於預覽階段）
2. 生成具有模型訪問權限的個人訪問令牌
3. 按上述方式配置環境變量

### 🏗️ **工作流程架構概述**

```mermaid
graph TD
    A[Workflow Builder] --> B[Agent Registry]
    B --> C[Workflow Execution Engine]
    C --> D[Agent 1: Content Generator]
    C --> E[Agent 2: Content Reviewer] 
    D --> F[Workflow Results]
    E --> F
    G[GitHub Models API] --> D
    G --> E
```

**主要組件:**
- **WorkflowBuilder**: 設計工作流程的主要編排引擎
- **AIAgent**: 具有特定能力的個別專業代理
- **GitHub Models Client**: AI 模型推理服務集成
- **Execution Context**: 管理工作流程階段間的狀態和數據流

## 🎨 **企業工作流程設計模式**

### 📝 **內容生產工作流程**
```
User Request → Content Generation → Quality Review → Final Output
```

### 🔍 **文檔處理管道**
```
Document Input → Analysis → Extraction → Validation → Structured Output
```

### 💼 **商業智能工作流程**
```
Data Collection → Processing → Analysis → Report Generation → Distribution
```

### 🤝 **客戶服務自動化**
```
Customer Inquiry → Classification → Processing → Response Generation → Follow-up
```

## 🏢 **企業效益**

### 🎯 **可靠性與可擴展性**
- **Deterministic Execution**: 一致且可重複的工作流程結果
- **Error Recovery**: 在任何工作流程階段都能優雅地處理故障
- **Performance Monitoring**: 跟蹤執行指標和優化機會
- **Resource Management**: 高效分配和利用 AI 模型資源

### 🔒 **安全性與合規性**
- **Secure Authentication**: 基於 GitHub 令牌的 API 訪問身份驗證
- **Audit Trails**: 完整記錄工作流程執行和決策點
- **Access Control**: 對工作流程執行和監控的細粒度權限控制
- **Data Privacy**: 在整個工作流程中安全處理敏感信息

### 📊 **可觀察性與管理**
- **Visual Workflow Design**: 清晰表示流程流和依賴關係
- **Execution Monitoring**: 實時跟蹤工作流程進度和性能
- **Error Reporting**: 詳細的錯誤分析和調試能力
- **Performance Analytics**: 用於優化和容量規劃的指標

讓我們開始構建您的首個企業級 AI 工作流程吧！🚀


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [2]:
#r "nuget: System.ClientModel, 1.6.1.0"

Installed Packages System.ClientModel, 1.6.1

In [3]:
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: OpenTelemetry.Api, 1.0.0"
#r "nuget: OpenTelemetry.Api, 1.0.0"

Installed Packages Azure.Identity, 1.15.0 OpenTelemetry.Api, 1.0.1 System.Linq.Async, 6.0.3

In [5]:

#r "nuget: Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3

In [ ]:

#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [9]:
using System;
using System.ComponentModel;
using System.ClientModel;
using OpenAI;
using Azure.Identity;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;

In [10]:
 using DotNetEnv;

In [11]:
Env.Load("../../../.env");

In [12]:

var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [13]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint = new Uri(github_endpoint)
};

In [14]:
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [15]:
const string ReviewerAgentName = "Concierge";
const string ReviewerAgentInstructions = @"
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. ";

In [16]:
const string FrontDeskAgentName = "FrontDesk";
const string FrontDeskAgentInstructions = @"""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """;

In [17]:
AIAgent reviewerAgent = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:ReviewerAgentName,instructions:ReviewerAgentInstructions);
AIAgent frontDeskAgent  = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:FrontDeskAgentName,instructions:FrontDeskAgentInstructions);

In [18]:
var workflow = new WorkflowBuilder(frontDeskAgent)
            .AddEdge(frontDeskAgent, reviewerAgent)
            .Build();

In [19]:
ChatMessage userMessage = new ChatMessage(ChatRole.User, [
	new TextContent("I would like to go to Paris.") 
]);

In [20]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, userMessage);

In [21]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));
string id="";
string messageData="";
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
    if (evt is AgentRunUpdateEvent executorComplete)
    {
        if(id=="")
        {
            id=executorComplete.ExecutorId;
        }
        if(id==executorComplete.ExecutorId)
        {
            messageData+=executorComplete.Data.ToString();
        }
        else
        {
            id=executorComplete.ExecutorId;
        }
        // Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
    }
}

Console.WriteLine(messageData);

Visit the Louvre Museum. It's a must-see for art enthusiasts and history lovers.That recommendation is quite popular and likely to attract many tourists. To refine it for a more local and authentic experience, consider suggesting an alternative that focuses on smaller, lesser-known art venues or galleries. Look for places where local artists exhibit or community spaces that host cultural events. This approach allows travelers to connect with the local art scene more intimately, away from the typical tourist routes.



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵信息，建議使用專業人工翻譯。我們對因使用此翻譯而產生的任何誤解或錯誤解釋不承擔責任。
